<a href="https://colab.research.google.com/github/Buketklfa/mushroom_final_project_Buket_Kalfa_/blob/main/notebook6eeeb89bca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'mushroom-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F478%2F974%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T073651Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2b18edd304340e47072c7ae40edff48f6956dab651f2234da19595cb679bb1883e7051499e11f87ebc5c853187d4f93a4cf1c19fe03e5775b0adc3efc0476785fdf85a112f61a6713bc5d91f1ec50a25a58f2a0e3cf33bfc10e59012145e69839373603b28cf9024a781db163c5f6f3847b34ae1ae69de11b2b2e2bf1958563acfc0b96840747485dd45f2b4f298c140fd270b8f7d57fffcb195043b18410a96ee5c5048e3c94d176f7441ded8cbbc95bb502208845f396d53ffeadb33a1b7f5f69778777168398bef6e34f177819a86d4ca573041603d2a5d12872295dbff0fc6daea51212dfdf13043c5002667744c0c609f988b74c01ab73a3af3995d5895'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

In [ ]:
# Veri setinin ilk 5 satırını görüntüleyelim
df.head()

# Veri seti hakkında genel bilgi alalım
df.info()

# Veri setindeki özet istatistikleri inceleyelim
df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Bağımlı değişkenin (class) dağılımı
sns.countplot(x='class', data=df)
plt.title('Sınıf Dağılımı (Yenilebilir/Zehirli)')
plt.show()

# Diğer kategorik değişkenlerin bazılarını inceleyelim
plt.figure(figsize=(12,6))
sns.countplot(x='odor', data=df, hue='class')
plt.title('Koku Özelliği ile Sınıf Dağılımı')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Tüm kategorik değişkenleri sayısal hale getirmek için Label Encoding
label_encoder = LabelEncoder()

for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

# Veri kümesini eğitim ve test olarak bölme
from sklearn.model_selection import train_test_split

X = df.drop('class', axis=1)  # Bağımsız değişkenler
y = df['class']  # Bağımlı değişken (yenilebilir/zehirli)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# İterasyon sayısını artırarak uyarıyı çözebiliriz
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Model performansı
y_pred = model.predict(X_test)
print('Doğruluk Skoru:', accuracy_score(y_test, y_pred))
print('Sınıflandırma Raporu:')
print(classification_report(y_test, y_pred))from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# K-Means kümeleme (n_clusters=2 çünkü mantarlar iki sınıfa ayrılıyor)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

# Küme etiketlerini ekleyelim
df['cluster'] = kmeans.labels_

# Kümeleme sonuçlarını inceleyelim
sns.countplot(x='cluster', data=df)
plt.title('K-Means Kümeleme Sonuçları')
plt.show()

# Kümelerin sınıflarla nasıl eşleştiğini görelim
ct = pd.crosstab(df['class'], df['cluster'])
print(ct)

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# K-Means kümeleme (n_clusters=2 çünkü mantarlar iki sınıfa ayrılıyor)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

# Küme etiketlerini ekleyelim
df['cluster'] = kmeans.labels_

# Kümeleme sonuçlarını inceleyelim
sns.countplot(x='cluster', data=df)
plt.title('K-Means Kümeleme Sonuçları')
plt.show()

# Kümelerin sınıflarla nasıl eşleştiğini görelim
ct = pd.crosstab(df['class'], df['cluster'])
print(ct)

In [ ]:
# K-Means kümeleme (n_clusters=2, n_init=10 olarak ayarlandı)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans.fit(X)

# Küme etiketlerini ekleyelim
df['cluster'] = kmeans.labels_

# Kümeleme sonuçlarını inceleyelim
sns.countplot(x='cluster', data=df)
plt.title('K-Means Kümeleme Sonuçları')
plt.show()

# Kümelerin sınıflarla nasıl eşleştiğini görelim
ct = pd.crosstab(df['class'], df['cluster'])
print(ct)